---

_You are currently looking at **version 1.5** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 3 - More Pandas
This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [65]:
import numpy as np
import pandas as pd

def answer_one():
    url='http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls'
    energy = pd.read_excel('Energy Indicators.xls')
    energy = energy.drop(energy.columns[[0, 1]], axis=1)
    mapping = {energy.columns[0]:'Country', energy.columns[1]: 'Energy Supply', energy.columns[2]:'Energy Supply per Capita', energy.columns[3]: '% Renewable'}
    energy = energy.rename(columns=mapping)
    energy=energy.dropna(axis=0).drop(7)
    energy=energy.reset_index()
    energy['Energy Supply']=1000000*energy['Energy Supply']
    
    #先整理資料刪除多於字元
    energy["Country"] = energy["Country"].str.strip("0123456789").str.replace(r"(\(.*?\))", "").str.strip().replace("...",np.nan)
    
    #再rename index,否則會連index一起整理刪掉必要資訊
    energy=energy.set_index('Country')
    energy=energy.rename(index={"Republic of Korea": "South Korea",
        "United States of America": "United States",
        "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
        "China, Hong Kong Special Administrative Region": "Hong Kong"}).reset_index()
    energy=energy.drop(['index'],axis=1)

   

    url1='http://data.worldbank.org/indicator/NY.GDP.MKTP.CD' 
    GDP=pd.read_csv('world_bank.csv',skiprows=4)
    #GDP = GDP.drop([0,1,2]) 
    #GDP.columns = GDP.columns.astype(str)
    #GDP.columns = GDP.loc[3]
    GDP=GDP.rename(columns={'Country Name': 'Country'}).set_index('Country')
    GDP=GDP.rename(index={"Korea, Rep.": "South Korea","Iran, Islamic Rep.": "Iran","Hong Kong SAR, China": "Hong Kong"})
    columns_to_keep = [ 'Country','2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
    GDP=GDP.reset_index()
    GDP=GDP[columns_to_keep]
    
    
    url2='http://www.scimagojr.com/countryrank.php?category=2102'
    ScimEn = pd.read_excel('scimagojr-3.xlsx')
    ScimEn=ScimEn.iloc[0:15]
    
    
    df_new = pd.merge(ScimEn, energy, how='left', left_on='Country', right_on='Country')
    df_new=pd.merge(df_new,GDP,how='left', left_on='Country', right_on='Country').set_index('Country')
    
    return df_new
    
    
    #jcols = ['Country']
    #ucols = ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015'
    

            
answer_one()


,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,127191000000,93,19.7549,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
United States,2,96661,94747,792274,265436,8.20,230,90838000000,286,11.571,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
Japan,3,30504,30287,223024,61554,7.31,134,18984000000,149,10.2328,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
United Kingdom,4,20944,20357,206091,37874,9.84,139,7920000000,124,10.6005,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
Russian Federation,5,18534,18301,34266,12422,1.85,57,30709000000,214,17.2887,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12
Canada,6,17899,17620,215003,40930,12.01,149,10431000000,296,61.9454,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
Germany,7,17027,16831,140566,27426,8.26,126,13261000000,165,17.9015,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
India,8,15005,14841,128763,37209,8.58,115,33195000000,26,14.9691,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12
France,9,13153,12973,130632,28601,9.93,114,10597000000,166,17.0203,2.607840e+12,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12


### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [2]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [121]:
def answer_two():
    url='http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls'
    energy = pd.read_excel('Energy Indicators.xls')
    energy = energy.drop(energy.columns[[0, 1]], axis=1)
    mapping = {energy.columns[0]:'Country', energy.columns[1]: 'Energy Supply', energy.columns[2]:'Energy Supply per Capita', energy.columns[3]: '% Renewable'}
    energy = energy.rename(columns=mapping)
    energy=energy.dropna(axis=0).drop(7).set_index('Country').rename(index={"Republic of Korea": "South Korea",
        "United States of America": "United States",
        "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
        "China, Hong Kong Special Administrative Region": "Hong Kong"}).reset_index().replace("...",np.nan)
    energy['Energy Supply']=1000000*energy['Energy Supply']
    #energy['Country'] = energy['Country'].str.extract('([a-zA-Z]+)')
    #if Country.find('('):
            #energy.rename(index={country:country[:country.find('(')+1]}, inplace=True)
    energy['Country'] = energy['Country'].str.replace(r"\(.*\)","")
    
   

    url1='http://data.worldbank.org/indicator/NY.GDP.MKTP.CD' 
    GDP=pd.read_csv('world_bank.csv',skiprows=4)
    #GDP = GDP.drop([0,1,2]) 
    #GDP.columns = GDP.columns.astype(str)
    #GDP.columns = GDP.loc[3]
    GDP=GDP.rename(columns={'Country Name': 'Country'}).set_index('Country')
    GDP=GDP.rename(index={"Korea, Rep.": "South Korea", 
        "Iran, Islamic Rep.": "Iran",
        "Hong Kong SAR, China": "Hong Kong"})
    columns_to_keep = [ 'Country','2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
    GDP=GDP.reset_index()
    GDP=GDP[columns_to_keep]
    
    
    url2='http://www.scimagojr.com/countryrank.php?category=2102'
    ScimEn = pd.read_excel('scimagojr-3.xlsx')
    ScimEn=ScimEn.iloc[15:]
    
    
    df_new = pd.merge(ScimEn, energy, how='outer', left_on='Country', right_on='Country')
    df_new=pd.merge(df_new,GDP,how='outer', left_on='Country', right_on='Country').set_index('Country')
    
    return df_new

    
answer_two()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
Taiwan,16.0,6468.0,6441.0,86099.0,14768.0,13.31,102.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Turkey,17.0,5879.0,5827.0,91857.0,23165.0,15.62,104.0,4.897000e+09,64.0,28.536690,6.680034e+11,6.991896e+11,7.037962e+11,6.698319e+11,7.311681e+11,7.953116e+11,8.122315e+11,8.462844e+11,8.718432e+11,9.065849e+11
Norway,18.0,5675.0,5634.0,32693.0,6024.0,5.76,74.0,1.388000e+09,273.0,97.635580,4.190512e+11,4.313284e+11,4.329860e+11,4.259607e+11,4.285247e+11,4.326762e+11,4.445694e+11,4.490094e+11,4.589539e+11,4.662883e+11
Netherlands,19.0,5170.0,5105.0,48351.0,6067.0,9.35,85.0,NaN,NaN,NaN,8.127830e+11,8.428435e+11,8.571640e+11,8.248696e+11,8.364397e+11,8.503550e+11,8.413664e+11,8.371985e+11,8.456636e+11,8.624973e+11
Sweden,20.0,4565.0,4494.0,57889.0,9632.0,12.68,95.0,2.044000e+09,212.0,46.598460,4.726099e+11,4.887020e+11,4.859797e+11,4.607834e+11,4.883793e+11,5.013917e+11,4.999562e+11,5.061616e+11,5.176362e+11,5.388264e+11
Malaysia,21.0,3914.0,3881.0,25545.0,6597.0,6.53,62.0,3.417000e+09,116.0,7.754589,2.163030e+11,2.299275e+11,2.410371e+11,2.373885e+11,2.550169e+11,2.685170e+11,2.832141e+11,2.965633e+11,3.143352e+11,3.299022e+11
Switzerland,22.0,3636.0,3599.0,38962.0,6186.0,10.72,81.0,NaN,NaN,NaN,5.415550e+11,5.639773e+11,5.768204e+11,5.645363e+11,5.812117e+11,5.917000e+11,5.983544e+11,6.089383e+11,6.204440e+11,6.260878e+11
Denmark,23.0,3531.0,3494.0,37254.0,6191.0,10.55,85.0,NaN,NaN,NaN,3.312336e+11,3.339644e+11,3.315667e+11,3.146968e+11,3.198110e+11,3.234957e+11,3.232585e+11,3.224704e+11,3.265397e+11,3.303882e+11
Mexico,24.0,3249.0,3214.0,21524.0,4145.0,6.62,57.0,7.944000e+09,64.0,12.913400,1.002729e+12,1.034857e+12,1.049348e+12,1.000025e+12,1.051129e+12,1.093643e+12,1.137587e+12,1.152892e+12,1.178836e+12,1.208859e+12


## Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [2]:
def answer_three():
   
    Top15 = answer_one()
    #Top15.drop(['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable'], axis=1).apply(lambda x: x.mean(), axis=1)
    columns_to_keep = [ 'Country','2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
    Top15=Top15.reset_index().replace("...",np.nan)
    Top15=Top15[columns_to_keep].set_index('Country')
    
    
    rows = ['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
    Top15=Top15.apply(lambda x: np.mean(x[rows]), axis=1)
    
    avgGDP=Top15
    #avgGDP.rename(columns = {0:'avg'},inplace=True)
    avgGDP1=avgGDP.sort_values(ascending=False)
   
    return avgGDP1
    
    
    
answer_three()

Country
United States         1.536434e+13
China                 6.348609e+12
Japan                 5.542208e+12
Germany               3.493025e+12
France                2.681725e+12
United Kingdom        2.487907e+12
Brazil                2.189794e+12
Italy                 2.120175e+12
India                 1.769297e+12
Canada                1.660647e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
Australia             1.164043e+12
South Korea           1.106715e+12
Iran                  4.441558e+11
dtype: float64

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [3]:
def answer_four():
    
    Top15 = answer_one()
    avgGDP=answer_three()
    
    C4=Top15.loc['United Kingdom']
    answer = C4['2015']-C4['2006']
    
    return answer

answer_four()

246702696075.3999

### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [4]:
def answer_five():
    Top15 = answer_one()
    
    a=Top15["Energy Supply per Capita"].mean()
    return float(a)
    
    
answer_five()

157.6

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [5]:
def answer_six():
    Top15 = answer_one()
    
    
    #Top15=Top15['% Renewable'].idxmax(), Top15['% Renewable'].max()
    a=tuple(Top15.loc[[Top15['% Renewable'].idxmax()], '% Renewable'].reset_index().values[0])
    return a

answer_six()

('Brazil', 69.64803)

### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [6]:
def answer_seven():
    Top15 = answer_one()
    Top15['Cratio']=Top15['Self-citations']/Top15['Citations']
    a=tuple(Top15.loc[[Top15['Cratio'].idxmax()], 'Cratio'].reset_index().values[0])
    return a
answer_seven()

('China', 0.6893126179389422)

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [7]:
import heapq
def answer_eight():
    Top15 = answer_one()
    Top15['Ce']=Top15['Energy Supply']/Top15['Energy Supply per Capita']
    a=Top15['Ce'] .sort_values(ascending=False)
    a=a.reset_index().iloc[2]["Country"]
    
    
    
    
    #You can use nlargest function and then idxmin function afterwards.
    return a

answer_eight()

'United States'

### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [8]:
def answer_nine():
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15=Top15.loc[:,['Citable docs per Capita','Energy Supply per Capita']]
    return Top15['Citable docs per Capita'].astype('float64').corr(Top15['Energy Supply per Capita'].astype('float64'), method = "pearson")
    
    
answer_nine()

0.79400104354429457

In [52]:
#def plot9():
    #import matplotlib as plt
    #%matplotlib inline
    
    #Top15 = answer_one()
    #Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    #Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    #Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])
    
#plot9()

In [ ]:
#plot9() # Be sure to comment out plot9() before submitting the assignment!

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [9]:
def answer_ten():
    Top15 = answer_one()
    Top15['renew']=np.where(Top15['% Renewable']>=Top15['% Renewable'].median(),1,0)
    HighRenew=Top15.sort_values(by='Rank')['% Renewable']

    return HighRenew




answer_ten()

Country
China                 19.7549
United States          11.571
Japan                 10.2328
United Kingdom        10.6005
Russian Federation    17.2887
Canada                61.9454
Germany               17.9015
India                 14.9691
France                17.0203
South Korea           2.27935
Italy                 33.6672
Spain                 37.9686
Iran                  5.70772
Australia             11.8108
Brazil                 69.648
Name: % Renewable, dtype: object

### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [66]:
def answer_eleven():
    Top15 = answer_one()
    Top15['PopEst'] = (Top15['Energy Supply'] / Top15['Energy Supply per Capita']).astype('float64')
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    s = pd.Series(ContinentDict).reset_index()
    s.columns = ['Country', 'Continent']
    
    Top15 = Top15.reset_index()
    
    df=pd.merge(Top15, s, how='left', left_on='Country', right_on='Country')
    df=df.loc[:,['PopEst','Continent','Country']]
    
    
    df=df.set_index('Continent').groupby(level=0)['PopEst'].agg({'size': np.size,'sum': np.sum,'mean': np.average, 'std': np.nanstd})
    
    
    
    return df

answer_eleven()

,size,sum,mean,std
Continent,,,,
Asia,5.0,2.898666e+09,5.797333e+08,6.074036e+08
Australia,1.0,2.331602e+07,2.331602e+07,0.000000e+00
Europe,6.0,4.579297e+08,7.632161e+07,3.162885e+07
North America,2.0,3.528552e+08,1.764276e+08,1.411878e+08
South America,1.0,2.059153e+08,2.059153e+08,0.000000e+00


### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [ ]:

def answer_twelve():
    Top15 = answer_one()
    return "ANSWER"

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [ ]:
def answer_thirteen():
    Top15 = answer_one()
    return "ANSWER"

### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [ ]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [ ]:
#plot_optional() # Be sure to comment out plot_optional() before submitting the assignment!